In [2]:
import pandas as pd
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import shape
import json
from shapely.geometry import Point

import statsmodels.api as sm

In [3]:
df=pd.read_excel('https://raw.githubusercontent.com/BiancaDiFranco/M-todosOP/main/Trabajo%20final/Rosario%20F.xlsx', engine='openpyxl')

In [4]:
dummies = pd.get_dummies(df["INT DE VOTO X ESPACIO"], prefix="INT DE VOTO X ESPACIO")
df = pd.concat([df, dummies], axis=1)

In [5]:
df["GENERO"] = (df["GENERO"] == "Femenino").astype(int)
df['GENERO']

0      0
1      0
2      1
3      0
4      1
      ..
367    0
368    1
369    0
370    0
371    1
Name: GENERO, Length: 372, dtype: int32

In [6]:
dummies = pd.get_dummies(df["EDAD"], prefix="EDAD")
df = pd.concat([df, dummies], axis=1)

In [7]:
df['INT DE VOTO X ESPACIO_LIBERTARIOS']

0       True
1      False
2      False
3       True
4      False
       ...  
367    False
368    False
369     True
370    False
371    False
Name: INT DE VOTO X ESPACIO_LIBERTARIOS, Length: 372, dtype: bool

In [8]:
estratos=gpd.read_file('https://github.com/BiancaDiFranco/M-todosOP/raw/refs/heads/main/Trabajo%20final/datasets%20complementarios/poblaciones/Estratificaci%C3%B3n%20de%20radios%20censales%20-%20Rosario.csv')
estratos['Geometría en GeoJSON'] = estratos['Geometría en GeoJSON'].apply(lambda x: shape(json.loads(x)))
estratos=estratos.set_geometry('Geometría en GeoJSON')
estratos=gpd.GeoDataFrame(estratos, geometry='Geometría en GeoJSON', crs='EPSG:4326')
estratos = estratos.dropna(subset=['Latitud del centroide', 'Longitud del centroide'])
estratos = estratos.dropna(subset=["Segmento socioeconómico del radio"])
estratos = estratos.dropna(subset=['Geometría en GeoJSON'])

In [9]:
estratos

,Código de radio,Cantidad total de personas,Cantidad total de hogares,Hogares con al menos un indicador NBI,Hogares con computadora,Hogares sin heladera,Cantidad de personas en el hogar,Población de 3 a 4 años,Población de 14 a 25 años,Población de 18 a 65 años,...,Código de provincia,Nombre de provincia,Código de departamentos/comuna,Nombre de departamentos/comuna,Población total,Total de hogares,Latitud del centroide,Longitud del centroide,Geometría en GeoJSON,geometry
0,820840421,1800,542,73,290,109,3.32103321,67,386,1118,...,82,Santa Fe,82084,Rosario,1800,542,-32.87955,-60.713932,"POLYGON ((-60.71118 -32.88183, -60.71097 -32.8...",None
1,820840403,989,345,2,222,1,2.86666667,18,160,654,...,82,Santa Fe,82084,Rosario,995,345,-32.872321,-60.693321,"POLYGON ((-60.69067 -32.87038, -60.69030 -32.8...",None
2,820840301,484,136,43,13,54,3.55882353,31,109,232,...,82,Santa Fe,82084,Rosario,484,136,-32.886938,-60.755944,"POLYGON ((-60.75295 -32.87326, -60.74423 -32.8...",None
3,820840302,1541,395,74,115,31,3.90126582,68,409,901,...,82,Santa Fe,82084,Rosario,1541,395,-32.875657,-60.735103,"POLYGON ((-60.73152 -32.87805, -60.73144 -32.8...",None
4,820840401,534,162,5,91,2,3.2962963,11,105,347,...,82,Santa Fe,82084,Rosario,534,162,-32.874749,-60.709626,"POLYGON ((-60.69970 -32.87109, -60.69965 -32.8...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,820845522,1620,439,130,59,89,3.69020501,78,448,865,...,82,Santa Fe,82084,Rosario,1620,439,-33.019926,-60.649603,"POLYGON ((-60.64566 -33.01662, -60.64542 -33.0...",None
1066,820845206,1453,371,32,99,25,3.91644205,58,348,853,...,82,Santa Fe,82084,Rosario,1453,371,-33.0206,-60.660675,"POLYGON ((-60.66040 -33.02182, -60.66039 -33.0...",None
1067,820845207,1664,444,105,56,83,3.74774775,86,469,904,...,82,Santa Fe,82084,Rosario,1664,444,-33.022212,-60.660949,"POLYGON ((-60.65852 -33.02206, -60.65841 -33.0...",None
1068,820844814,1195,309,56,84,19,3.86731392,39,318,726,...,82,Santa Fe,82084,Rosario,1195,309,-33.027272,-60.68357,"POLYGON ((-60.68154 -33.02739, -60.68176 -33.0...",None


In [10]:
geometry = [Point(xy) for xy in zip(df['Coord1__Longitude'], df['Coord1__Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.set_crs('EPSG:4326', inplace=True)
union=gpd.sjoin(gdf, estratos, how='inner', predicate='within')
union['Hogares con hacinamiento (>= 3 personas por cuarto)']

0       0
1       0
2      15
3      15
4      21
       ..
367     1
368     0
369     6
370     0
371     0
Name: Hogares con hacinamiento (>= 3 personas por cuarto), Length: 328, dtype: object

In [11]:
union['Segmento socioeconómico del radio']=union['Segmento socioeconómico del radio'].astype(int)
union['Hogares con jefatura femenina']= union['Hogares con jefatura femenina'].astype(int)

In [12]:
X = union[['GENERO','Hogares con jefatura femenina']]
y = union['INT DE VOTO X ESPACIO_LIBERTARIOS']    

In [13]:
X = sm.add_constant(X)

In [14]:
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.403190
         Iterations 7
                                   Logit Regression Results                                  
Dep. Variable:     INT DE VOTO X ESPACIO_LIBERTARIOS   No. Observations:                  328
Model:                                         Logit   Df Residuals:                      325
Method:                                          MLE   Df Model:                            2
Date:                               Tue, 26 Nov 2024   Pseudo R-squ.:                 0.05558
Time:                                       18:41:31   Log-Likelihood:                -132.25
converged:                                      True   LL-Null:                       -140.03
Covariance Type:                           nonrobust   LLR p-value:                 0.0004171
                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------

In [15]:
union['Hogares con al menos un indicador NBI']=union['Hogares con al menos un indicador NBI'].astype(int)
union['Hogares con hacinamiento (>= 3 personas por cuarto)']=union['Hogares con hacinamiento (>= 3 personas por cuarto)'].astype(int)
union['Hogares con indicadores de alta marginalidad']=union['Hogares con indicadores de alta marginalidad'].astype(int)
union['Población de 18 a 29 que no estudia ni trabaja']=union['Población de 18 a 29 que no estudia ni trabaja'].astype(int)
union['Jefes sin secundaria completa']=union['Jefes sin secundaria completa'].astype(int)
union['Hogares con computadora']=union['Hogares con computadora'].astype(int)

In [16]:
X = union[['Hogares con al menos un indicador NBI','Hogares con hacinamiento (>= 3 personas por cuarto)','Hogares con indicadores de alta marginalidad','Segmento socioeconómico del radio','Población de 18 a 29 que no estudia ni trabaja']]
y = union['INT DE VOTO X ESPACIO_FdT']   
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.522519
         Iterations 6
                               Logit Regression Results                              
Dep. Variable:     INT DE VOTO X ESPACIO_FdT   No. Observations:                  328
Model:                                 Logit   Df Residuals:                      322
Method:                                  MLE   Df Model:                            5
Date:                       Tue, 26 Nov 2024   Pseudo R-squ.:                 0.02135
Time:                               18:41:31   Log-Likelihood:                -171.39
converged:                              True   LL-Null:                       -175.12
Covariance Type:                   nonrobust   LLR p-value:                    0.1875
                                                          coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------

In [17]:
X = union[['Hogares con al menos un indicador NBI','Hogares con hacinamiento (>= 3 personas por cuarto)','Hogares con indicadores de alta marginalidad','Segmento socioeconómico del radio','Población de 18 a 29 que no estudia ni trabaja']]
y = union['INT DE VOTO X ESPACIO_FIT']   
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.123782
         Iterations 12
                               Logit Regression Results                              
Dep. Variable:     INT DE VOTO X ESPACIO_FIT   No. Observations:                  328
Model:                                 Logit   Df Residuals:                      322
Method:                                  MLE   Df Model:                            5
Date:                       Tue, 26 Nov 2024   Pseudo R-squ.:                 0.09273
Time:                               18:41:31   Log-Likelihood:                -40.601
converged:                              True   LL-Null:                       -44.750
Covariance Type:                   nonrobust   LLR p-value:                    0.1405
                                                          coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [18]:
X = union[['Segmento socioeconómico del radio','Población de 18 a 29 que no estudia ni trabaja','Jefes sin secundaria completa','Hogares con computadora']]
y = union['INT DE VOTO X ESPACIO_LIBERTARIOS']   
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.423780
         Iterations 6
                                   Logit Regression Results                                  
Dep. Variable:     INT DE VOTO X ESPACIO_LIBERTARIOS   No. Observations:                  328
Model:                                         Logit   Df Residuals:                      323
Method:                                          MLE   Df Model:                            4
Date:                               Tue, 26 Nov 2024   Pseudo R-squ.:                0.007348
Time:                                       18:41:31   Log-Likelihood:                -139.00
converged:                                      True   LL-Null:                       -140.03
Covariance Type:                           nonrobust   LLR p-value:                    0.7251
                                                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------

In [19]:
elecciones2015=gpd.read_file('https://github.com/BiancaDiFranco/M-todosOP/raw/refs/heads/main/Trabajo%20final/datasets%20complementarios/Elecciones%20presidenciales,%202015%20-%20Rosario.csv')

In [20]:
elecciones2015['Geometría en GeoJSON'] = elecciones2015['Geometría en GeoJSON'].apply(lambda x: shape(json.loads(x)))
elecciones2015=elecciones2015.set_geometry('Geometría en GeoJSON')
elecciones2015=gpd.GeoDataFrame(elecciones2015, geometry='Geometría en GeoJSON', crs='EPSG:4326')
elecciones2015 = elecciones2015.dropna(subset=['Latitud del centroide', 'Longitud del centroide'])
elecciones2015 = elecciones2015.dropna(subset=['Fórmula con más votos en el radio'])
elecciones2015 = elecciones2015.dropna(subset=['Geometría en GeoJSON'])

In [21]:
geometry = [Point(xy) for xy in zip(df['Coord1__Longitude'], df['Coord1__Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.set_crs('EPSG:4326', inplace=True)
union2015=gpd.sjoin(gdf, elecciones2015, how='inner', predicate='within')

In [22]:
union2015['Sergio Massa (UNA)']=union2015['Sergio Massa (UNA)'].astype(float)
union2015['Maurio Macri (Cambiemos)']=union2015['Maurio Macri (Cambiemos)'].astype(float)

In [23]:
X = union2015[['Maurio Macri (Cambiemos)']]
y = union2015['INT DE VOTO X ESPACIO_JxC']   
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.550217
         Iterations 6
                               Logit Regression Results                              
Dep. Variable:     INT DE VOTO X ESPACIO_JxC   No. Observations:                  327
Model:                                 Logit   Df Residuals:                      325
Method:                                  MLE   Df Model:                            1
Date:                       Tue, 26 Nov 2024   Pseudo R-squ.:                 0.01107
Time:                               18:41:39   Log-Likelihood:                -179.92
converged:                              True   LL-Null:                       -181.94
Covariance Type:                   nonrobust   LLR p-value:                   0.04471
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [24]:
union.columns

Index(['interview__key', 'METODO', 'Coord1__Latitude', 'Coord1__Longitude',
       'EDAD', 'GENERO', 'INT DE VOTO X ESPACIO', 'RADIO',
       'INT DE VOTO X ESPACIO_FIT', 'INT DE VOTO X ESPACIO_FdT',
       'INT DE VOTO X ESPACIO_JxC', 'INT DE VOTO X ESPACIO_LIBERTARIOS',
       'INT DE VOTO X ESPACIO_NINGUNO/NOSABE', 'INT DE VOTO X ESPACIO_OTROS',
       'INT DE VOTO X ESPACIO_PNK', 'EDAD_16 a 25', 'EDAD_26 a 35',
       'EDAD_36 a 45', 'EDAD_46 a 55', 'EDAD_56 y mas', 'geometry_left',
       'index_right', 'Código de radio', 'Cantidad total de personas',
       'Cantidad total de hogares', 'Hogares con al menos un indicador NBI',
       'Hogares con computadora', 'Hogares sin heladera',
       'Cantidad de personas en el hogar', 'Población de 3 a 4 años',
       'Población de 14 a 25 años', 'Población de 18 a 65 años',
       'Hogares con hacinamiento (>= 3 personas por cuarto)',
       'Hogares sin cañería de agua en la vivienda',
       'Hogares sin cloaca / red pública',
       'H

In [25]:
union['Población de 18 y más hasta primaria']=union['Población de 18 y más hasta primaria'].astype(int)
union['Población de 18 y más con primaria y secundario completo']=union['Población de 18 y más con primaria y secundario completo'].astype(int)
union['Población de 18 y más con secundario completo o terciario sin universitario completo']=union['Población de 18 y más con secundario completo o terciario sin universitario completo'].astype(int)
union['Población con universitario completo o más']=union['Población con universitario completo o más'].astype(int)

In [26]:
X = union[['Población de 18 y más con primaria y secundario completo','Población de 18 y más hasta primaria','Población de 18 y más con secundario completo o terciario sin universitario completo', 'Población con universitario completo o más']]
y = union['INT DE VOTO X ESPACIO_JxC']
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.551514
         Iterations 6
                               Logit Regression Results                              
Dep. Variable:     INT DE VOTO X ESPACIO_JxC   No. Observations:                  328
Model:                                 Logit   Df Residuals:                      323
Method:                                  MLE   Df Model:                            4
Date:                       Tue, 26 Nov 2024   Pseudo R-squ.:                0.007241
Time:                               18:41:39   Log-Likelihood:                -180.90
converged:                              True   LL-Null:                       -182.22
Covariance Type:                   nonrobust   LLR p-value:                    0.6200
                                                                                           coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------

In [27]:
X = union[['Población de 18 y más hasta primaria','Población de 18 y más con primaria y secundario completo','Población de 18 y más con secundario completo o terciario sin universitario completo', 'Población con universitario completo o más']]
y = union['INT DE VOTO X ESPACIO_FdT']
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.530464
         Iterations 5
                               Logit Regression Results                              
Dep. Variable:     INT DE VOTO X ESPACIO_FdT   No. Observations:                  328
Model:                                 Logit   Df Residuals:                      323
Method:                                  MLE   Df Model:                            4
Date:                       Tue, 26 Nov 2024   Pseudo R-squ.:                0.006468
Time:                               18:41:40   Log-Likelihood:                -173.99
converged:                              True   LL-Null:                       -175.12
Covariance Type:                   nonrobust   LLR p-value:                    0.6871
                                                                                           coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------

In [28]:
X = union[['Población de 18 y más hasta primaria','Población de 18 y más con primaria y secundario completo','Población de 18 y más con secundario completo o terciario sin universitario completo', 'Población con universitario completo o más']]
y = union['INT DE VOTO X ESPACIO_NINGUNO/NOSABE']
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.492772
         Iterations 6
                                    Logit Regression Results                                    
Dep. Variable:     INT DE VOTO X ESPACIO_NINGUNO/NOSABE   No. Observations:                  328
Model:                                            Logit   Df Residuals:                      323
Method:                                             MLE   Df Model:                            4
Date:                                  Tue, 26 Nov 2024   Pseudo R-squ.:                 0.01021
Time:                                          18:41:40   Log-Likelihood:                -161.63
converged:                                         True   LL-Null:                       -163.30
Covariance Type:                              nonrobust   LLR p-value:                    0.5034
                                                                                           coef    std err          z     

In [29]:
X = union[['Población de 18 y más hasta primaria','Población de 18 y más con primaria y secundario completo','Población de 18 y más con secundario completo o terciario sin universitario completo', 'Población con universitario completo o más']]
y = union['INT DE VOTO X ESPACIO_LIBERTARIOS']
X = sm.add_constant(X)
model = sm.Logit(y, X)
results = model.fit()  

print(results.summary())

Optimization terminated successfully.
         Current function value: 0.424445
         Iterations 6
                                   Logit Regression Results                                  
Dep. Variable:     INT DE VOTO X ESPACIO_LIBERTARIOS   No. Observations:                  328
Model:                                         Logit   Df Residuals:                      323
Method:                                          MLE   Df Model:                            4
Date:                               Tue, 26 Nov 2024   Pseudo R-squ.:                0.005789
Time:                                       18:41:40   Log-Likelihood:                -139.22
converged:                                      True   LL-Null:                       -140.03
Covariance Type:                           nonrobust   LLR p-value:                    0.8050
                                                                                           coef    std err          z      P>|z|      [0.025      